#### CellStrat Prompt Engineering for Generative AI - Module 2 : Applications Of Prompt Engineering

This notebook demonstrates prompt fine tunning for supply chain domain and provides prompt templates for real business applications

In [1]:
import textwrap

import google.generativeai as genai
import yaml
from yaml import SafeLoader

import os
from dotenv import load_dotenv

from IPython.display import display
from IPython.display import Markdown

c:\Users\Poulami\Desktop\Prompt Engg\PromptTemp\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [4]:
prompt = '''**Task:** Classify the following text into one of the following categories:
A: {A} 
B: {B} 
C: {C} 
**Text:** {text}

**Please select the most appropriate category.**
'''

In [6]:
 # Read the prompt template
with open(r"yaml/prompt.yaml", "r") as f:
    template = f.read()

In [7]:
# Fill placeholders with data
def classify_text(template,text, options, model_name="gemini-pro"):
  """
  Classifies text using a formatted prompt and Gemini Pro model.

  Args:
      text: The text to be classified.
      options: A dictionary containing classification options (A, B, C, etc.).
      model_name: The name of the Gemini Pro model (defaults to "gemini-pro").

  Returns:
      The predicted category (A, B, C, etc.).
  """

#   # Configure the API key
#   genai.configure(api_key=api_key)

  # Build the prompt with formatted options

  #prompt = prompt.format(text=text, **options)
  prompt = template.format(text=text, **options)
  print(prompt) 
  # Create the GenerativeModel object
  model = genai.GenerativeModel('gemini-1.0-pro')

  # Generate the content (classification response)
  response = model.generate_content(prompt)

  # Extract the predicted category (assuming format "Predicted category: X")
  predicted_category = response.text.split(": ")[-1].strip()
  #predicted_category = response
  return predicted_category

In [8]:
#text = "Sunny side up is a good recipe"
text = "It is sunny today"
options = {"A": "Sports", "B": "Outdoors", "C": "Weather"}

predicted_category = classify_text(template,text, options)

print(f"Predicted Category: {predicted_category}")

prompt = '''**Task:** Classify the following text into one of the following categories:
A: Sports 
B: Outdoors 
C: Weather 
**Text:** It is sunny today
#**Constraint: Do not predict anything other than the options provided, if none of it matches return the word None** 
#**Explanation: Return appropriate category with explanation on why a category has been chosen**
** Return an appropriate category**

'''
Predicted Category: Weather**
**Explanation:** The text "It is sunny today" describes a weather condition, which is a characteristic of the weather category.


In [9]:
### Real example template###
cs_prompt='''**Task:**Given an email containing email body and email subject classify the following text into one of the following categories:
{cat_names}
**The category descriptions are as follows:{category_descriptions} 
**Text:** {text}
#**Constraint: Return only the category name.Do not predict anything other than the options provided, if none of it matches return the word None** 
**Explanation: Return appropriate category from the above list with explanation on why a category has been chosen**'''


In [11]:
with open(r"yaml/order_prompt_template.yaml", "r") as f:
    order_template = f.read()

In [16]:
def classify_text_real(template,email, cat_names,category_description, model_name="gemini-1.0-pro"):
  """
  Classifies text using a formatted prompt and Gemini Pro model.

  Args:
      text: The text to be classified.
      options: A dictionary containing classification options (A, B, C, etc.).
      model_name: The name of the Gemini Pro model (defaults to "gemini-pro").

  Returns:
      The predicted category (A, B, C, etc.).
  """


  # Build the prompt with formatted options

  prompt=cs_prompt.format(text=email,cat_names=cat_names,category_descriptions=category_description)
  #print(prompt) 
  # Create the GenerativeModel object
  model = genai.GenerativeModel(model_name)

  # Generate the content (classification response)
  response = model.generate_content(prompt)

  # Extract the predicted category (assuming format "Predicted category: X")
  predicted_category = response.text.split(": ")[-1].strip()
  #predicted_category = response
  return predicted_category

In [12]:
with open("config/category_refined.yaml", "r") as f:
    cat_details = yaml.load(f,Loader=SafeLoader)

In [13]:
cat_names=cat_details['categories']
category_description=cat_details['category_description']

In [14]:
with open(r'examples\order_discrepancy.txt') as f:
    email = f.readlines()

In [17]:
classify_text_real(order_template,email, cat_names,category_description)

'order discrepancy\nThe text is about a customer who received a shipment of table cloths that had black streaks along the outer edges. The customer is returning the entire shipment and requesting a credit for the cost of the order plus shipping and handling. This is an example of an order discrepancy because the customer is reporting a problem with the order.'